# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import string
import timeit
import sys
from collections import Counter

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# Function to print bold text
bold = lambda text: color.BOLD + text + color.END

# Punkt sentence tokenizer models 
# that help to detect sentence boundaries
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Library of stopwords from nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Model for Stemming and Lemmatization
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

# For POS Tagging
nltk.download(['averaged_perceptron_tagger'])

# import sklearn libraries
# from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain, MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from imblearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.externals import joblib

# To handle imbalanced data
from imblearn.over_sampling import SMOTE

# To help us stack models
from mlxtend.classifier import StackingClassifier

In [ ]:
def load_data(database_filepath):
    """Creates sql engine from database_filepath,\
       reads it in as a pandas dataframe
                 
    Args:
    -----------
    database_filepath: A str type, file path to sql database .db file
    
    Returns:
    --------
    X: Pandas dataframe of features (message text)
    y: Pandas dataframe of labels of the message
    y.columns: list of categories for the labels of message
    """
    # load data from database
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql_table(table_name=database_filepath.split('/')[-1].split('.')[0], con=engine)
    
    X = df['message']
    y = df.loc[:, 'related':]
    return X, y, y.columns

X, y, category_names = load_data('./DisasterResponse.db')

In [ ]:
X.head()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text: str):
    """
    Function: 
    ---------
    Processes the text string into word tokens for an NLP model,
    with stop words removed, lemmatized, and everything is lowercase
    
    Parameters:
    -----------
    text: A str type, the sentence you want to tokenize
    
    Returns:
    --------
    A list of word tokens
    """
    return [WordNetLemmatizer().lemmatize(token.strip(), pos='v') for token in word_tokenize(text.lower()) if token not in stopwords.words("english") and token not in list(string.punctuation)]

In [5]:
tokenize('The dispute has hurt the global economy, crimped U.S. exports, damaged American manufacturers and rattled corporate executives and small-business owners alike.')

['dispute',
 'hurt',
 'global',
 'economy',
 'crimp',
 'u.s.',
 'export',
 'damage',
 'american',
 'manufacturers',
 'rattle',
 'corporate',
 'executives',
 'small-business',
 'owners',
 'alike']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
print('=============================')  
print('Building Model:')
print('-----------------------------')

# Aggregate an ensemble of RandomForest classifier chains and feed them
# to the meta classifier
print('Creating ClassifierChains...')
chains = [ClassifierChain(base_estimator=RandomForestClassifier(), order='random', random_state=42) for _ in range(10)]

# Meta Classifier that will take the predictions
# of each output of the classifier chains and figure out
# the weight of each classifier in predicting labels
print('Adding Meta Classifier...')
meta_clf = MultiOutputClassifier(AdaBoostClassifier())

# Stack the base learners 
print('Stacking Meta Classifier on top of ClassifierChains...')
sclf = StackingClassifier(classifiers=chains,
                          meta_classifier=meta_clf)

# Resample dataset to be balanced
print('Initializing SMOTE to balance dataset...')
sm = SMOTE(random_state=42)

# Final Pipeline
print('Building Pipeline...')
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('tfidf_vect', TfidfVectorizer(tokenizer=tokenize)),
        ]))#,
#         ('starting_verb', StartingVerbExtractor())
    ])),
    ('smote', sm),
    ('sclf', sclf)
])

parameters = {
    'features__text_pipeline__tfidf_vect__ngram_range': ((1, 1), (1, 2)),
    'sclf__classifierchain-1__base_estimator__n_estimators': [2, 3],
    'sclf__classifierchain-2__base_estimator__n_estimators': [2, 3]
#     'features__text_pipeline__tfidf_vect__max_df': (0.5, 0.75, 1.0),
#     'features__text_pipeline__tfidf_vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf_vect__use_idf': (True, False),
#     'features__transformer_weights': (
#         {'text_pipeline': 1, 'starting_verb': 0.5},
#         {'text_pipeline': 0.5, 'starting_verb': 1},
#         {'text_pipeline': 0.8, 'starting_verb': 1},
#     )
}

model = GridSearchCV(pipeline, param_grid=parameters, cv=3)

# # Hyperparameters for tuning in GridSearch
# params = {
#     'features__text_pipeline__tfidf_vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__tfidf_vect__max_df': (0.5, 0.75, 1.0),
#     'features__text_pipeline__tfidf_vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf_vect__use_idf': (True, False),
#     'sclf__classifiers__classifierchain__base_estimator__randomforestclassifier__n_estimators': [100, 200],
#     'sclf__classifiers__classifierchain__base_estimator__randomforestclassifier__min_samples_split': [2, 5, 10],
#     'sclf__classifiers__classifierchain__base_estimator__adaboostclassifier__n_estimators': [100, 200],
#     'sclf__classifiers__classifierchain__base_estimator__adaboostclassifier__learning_rate': [0.01, 0.1, 1.0],
#     'sclf__classifiers__classifierchain__base_estimator__gradientboostingclassifier__n_estimators': [100, 200],
#     'sclf__classifiers__classifierchain__base_estimator__gradientboostingclassifier__learning_rate': [0.01, 0.1],
#     'sclf__classifiers__classifierchain__base_estimator__gradientboostingclassifier__min_samples_split': [2, 5, 10],
#     'sclf__classifiers__classifierchain__base_estimator__extratreesclassifier__n_estimators': [100, 200],
#     'sclf__classifiers__classifierchain__base_estimator__extratreesclassifier__min_samples_split': [2, 5, 10],
#     'sclf__classifiers__classifierchain__base_estimator__svc__C': [0.01, 0.1, 1.0],
#     'sclf__classifiers__classifierchain__base_estimator__svc__kernel': ['rbf', 'sigmoid'],
#     'sclf__classifiers__classifierchain__base_estimator__kneighborsclassifier__n_neighbors': [1, 5],
#     'sclf__classifiers__classifierchain__base_estimator__kneighborsclassifier__n_neighbors': [1, 5],
#     'sclf__meta_classifier__base_estimator__C': [0.1, 10.0],
#     'features__transformer_weights': (
#         {'text_pipeline': 1, 'starting_verb': 0.5},
#         {'text_pipeline': 0.5, 'starting_verb': 1},
#         {'text_pipeline': 0.8, 'starting_verb': 1},
#     )
# }

# print('Initializing GridSearchCV...')
# model = GridSearchCV(estimator=pipeline, 
#                      param_grid=params,
#                      cv=5,
#                      refit=True)

Building Model:
-----------------------------
Creating ClassifierChains...
Adding Meta Classifier...
Stacking Meta Classifier on top of ClassifierChains...
Building Pipeline...


In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
model.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__features', 'estimator__sclf', 'estimator__features__n_jobs', 'estimator__features__transformer_list', 'estimator__features__transformer_weights', 'estimator__features__verbose', 'estimator__features__text_pipeline', 'estimator__features__text_pipeline__memory', 'estimator__features__text_pipeline__steps', 'estimator__features__text_pipeline__verbose', 'estimator__features__text_pipeline__tfidf_vect', 'estimator__features__text_pipeline__tfidf_vect__analyzer', 'estimator__features__text_pipeline__tfidf_vect__binary', 'estimator__features__text_pipeline__tfidf_vect__decode_error', 'estimator__features__text_pipeline__tfidf_vect__dtype', 'estimator__features__text_pipeline__tfidf_vect__encoding', 'estimator__features__text_pipeline__tfidf_vect__input', 'estimator__features__text_pipeline__tfidf_vect__lowercase', 'estimator__features__text_pipeline__tfidf_vect__max_df', 'estimator__fe

In [9]:
print('=============================')  
print('Training Model:')
print('-----------------------------')
model.fit(X_train, y_train)

Training Model:
-----------------------------


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('tfidf_vect',
                                                                                         TfidfVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.float

In [10]:
cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(model.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (model.cv_results_[cv_keys[0]][r],
             model.cv_results_[cv_keys[1]][r] / 2.0,
             model.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % model.best_params_)
print('Training Accuracy: %.2f' % model.best_score_)

0.256 +/- 0.00 {'features__text_pipeline__tfidf_vect__ngram_range': (1, 1), 'sclf__classifierchain-1__base_estimator__n_estimators': 2}
0.269 +/- 0.00 {'features__text_pipeline__tfidf_vect__ngram_range': (1, 1), 'sclf__classifierchain-1__base_estimator__n_estimators': 3}
0.252 +/- 0.00 {'features__text_pipeline__tfidf_vect__ngram_range': (1, 2), 'sclf__classifierchain-1__base_estimator__n_estimators': 2}
0.268 +/- 0.00 {'features__text_pipeline__tfidf_vect__ngram_range': (1, 2), 'sclf__classifierchain-1__base_estimator__n_estimators': 3}
Best parameters: {'features__text_pipeline__tfidf_vect__ngram_range': (1, 1), 'sclf__classifierchain-1__base_estimator__n_estimators': 3}
Training Accuracy: 0.27


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = model.predict(X_test)

for idx, label in enumerate(category_names):
    print("Classification Report for {}:".format(bold(label)))
    print(
        classification_report(
            y_true=np.array(y_test)[:, idx], 
            y_pred=y_pred[:, idx]
        )
    )

Classification Report for related:
              precision    recall  f1-score   support

           0       0.60      0.45      0.51      1245
           1       0.84      0.91      0.87      3998

    accuracy                           0.80      5243
   macro avg       0.72      0.68      0.69      5243
weighted avg       0.78      0.80      0.79      5243

Classification Report for request:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      4352
           1       0.75      0.46      0.57       891

    accuracy                           0.88      5243
   macro avg       0.82      0.72      0.75      5243
weighted avg       0.87      0.88      0.87      5243

Classification Report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5243
   macro avg       0.50  

              precision    recall  f1-score   support

           0       0.86      0.95      0.90      3771
           1       0.82      0.61      0.70      1472

    accuracy                           0.85      5243
   macro avg       0.84      0.78      0.80      5243
weighted avg       0.85      0.85      0.85      5243

Classification Report for floods:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4812
           1       0.83      0.42      0.56       431

    accuracy                           0.95      5243
   macro avg       0.89      0.71      0.76      5243
weighted avg       0.94      0.95      0.94      5243

Classification Report for storm:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      4764
           1       0.66      0.40      0.50       479

    accuracy                           0.93      5243
   macro avg       0.80      0.69      0.73      5243
weighte

In [12]:
for average_type in ['micro', 'macro']:
    print('{} F1-score: {}'.format(average_type, f1_score(y_test, y_pred, average=average_type)))
    print('{} Precision: {}'.format(average_type, precision_score(y_test, y_pred, average=average_type)))
    print('{} Recall: {}'.format(average_type, recall_score(y_test, y_pred, average=average_type)))

accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

micro F1-score: 0.6088721162062091
micro Precision: 0.7449255161599443
micro Recall: 0.5148413510747185
macro F1-score: 0.2949373610199553
macro Precision: 0.47367953797005047
macro Recall: 0.23426002104518323
Accuracy: 0.9417954097526862


/Users/jeffchenchengyi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jeffchenchengyi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jeffchenchengyi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jeffchenchengyi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

---
## Resources

- [sklearn Classifier chain example](https://scikit-learn.org/stable/auto_examples/multioutput/plot_classifier_chain_yeast.html#sphx-glr-auto-examples-multioutput-plot-classifier-chain-yeast-py)
- [A Deep Dive Into Sklearn Pipelines](https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines)
- [Pipeline for Multi-Label Classification with One-vs-Rest Meta Classifier](http://queirozf.com/entries/scikit-learn-pipeline-examples#pipeline-for-multi-label-classification-with-one-vs-rest-meta-classifier)
- [Stacked Classification and GridSearch](http://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/)